In [1]:
import torch
import itertools
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
from torch.nn import Parameter
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
# 辅助函数，用于生成训练样例的最优解
# 来自 https://gist.github.com/mlalevic/6222750
def tsp_opt(points):
    """
    O(2^n*n^2)
    :param points: List of (x, y) points
    :return: Optimal solution
    """

    def length(x_coord, y_coord):
        return np.linalg.norm(np.asarray(x_coord) - np.asarray(y_coord))

    # Calculate all lengths
    all_distances = [[length(x, y) for y in points] for x in points]
    # Initial value - just distance from 0 to every other point + keep the track of edges
    A = {(frozenset([0, idx+1]), idx+1): (dist, [0, idx+1]) for idx, dist in enumerate(all_distances[0][1:])}
    cnt = len(points)
    for m in range(2, cnt):
        B = {}
        for S in [frozenset(C) | {0} for C in itertools.combinations(range(1, cnt), m)]:
            for j in S - {0}:
                # This will use 0th index of tuple for ordering, the same as if key=itemgetter(0) used
                B[(S, j)] = min([(A[(S-{j}, k)][0] + all_distances[k][j], A[(S-{j}, k)][1] + [j])
                                 for k in S if k != 0 and k != j])
        A = B
    res = min([(A[d][0] + all_distances[0][d[1]], A[d][1]) for d in iter(A)])
    return np.asarray(res[1])

In [3]:
# 首先，编写生成训练集的DATASET

class TSP_Dataset(Dataset):
    def __init__(self, data_size, citys_size):
        """
        data_size: int, 训练集大小
        citys_size: int, 每个训练数据包含多少城市，
        """
        super(TSP_Dataset, self).__init__()
        self.data_size = data_size
        self.citys_size = citys_size
        self.data = self.get_data()
    
    def __len__(self):
        return self.data_size

    def __getitem__(self, idx):
        return {'Point': self.data['Point'][idx], 'Slove':self.data['Slove'][idx]}

    def get_data(self):
        """
        随生成数据
        """
        point = np.random.rand(self.data_size, self.citys_size, 2)
        slove = [tsp_opt(citys) for citys in point]
        return dict(Point = point, Slove = slove)

In [4]:
# 然后是模型

# Encoder Model

class Encoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_layers, dropout, bidirectional):
        super(Encoder, self).__init__()

        # 用于接下来初始化h0, c0
        self.num_layers = num_layers*2 if bidirectional else num_layers

        #为了维持不管是否BRNN，输出的hidden_size 都等于hidden_dim, 这里的self.hidden_dim意味着建立LSTM的hidden_dim
        self.hidden_dim = hidden_dim//2 if bidirectional else hidden_dim

        self.lstm = nn.LSTM(
            input_size = embedding_dim, 
            hidden_size = self.hidden_dim, 
            num_layers = num_layers, 
            batch_first = True,
            dropout = dropout, 
            bidirectional = bidirectional)
        
        self.h0 = Parameter(torch.zeros(1), requires_grad=False)
        self.c0 = Parameter(torch.zeros(1), requires_grad=False)

    def forward(self, embedded_inputs):

        batch_size = embedded_inputs.size(0)
        h_0 = self.h0.unsqueeze(0).unsqueeze(0).repeat(self.num_layers,
                                                      batch_size,
                                                      self.hidden_dim)
        c_0 = self.c0.unsqueeze(0).unsqueeze(0).repeat(self.num_layers,
                                                      batch_size,
                                                      self.hidden_dim)

        output, (h_n, c_n) = self.lstm(embedded_inputs, (h_0, c_0))    
        return output, (h_n, c_n)

In [5]:
# Attention Layer

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()

        self.h_t_linear = nn.Linear(hidden_dim, hidden_dim)
        self.h_s_linear = nn.Conv1d(hidden_dim, hidden_dim, 1, 1)

        # F.tanh 被 deprecated 了，要用nn.Tanh
        # https://discuss.pytorch.org/t/torch-tanh-vs-torch-nn-functional-tanh/15897
        # https://stackoverflow.com/questions/56723486/why-arent-torch-functional-sigmoid-and-torch-nn-functional-relu-deprecated-like
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()

        self.V = Parameter(torch.FloatTensor(hidden_dim), requires_grad=True)
        # 不要用uniform_，那是就地操作
        nn.init.uniform(self.V, -1, 1)
        # 不要用alpha[visted] = float('-inf'),好像也是就地操作？
        # self.inf = self._inf.unsqueeze(1).expand(*mask_size) 是为了避免就地操作，避免a = a的情况
        self._inf = Parameter(torch.FloatTensor([float('-inf')]), requires_grad=False)

    def forward(self, h_s, h_t, visted):
        """
        h_s = encoder_output （N, L, Hidden）
        h_t = h_t   (N, Hidden)
        visted :访问过的城市TrueFalse Tensor (N, L)就是N个L
        """
        # score = V * tanh(W[h_s, h_t])
        
        #(N, Hidden, L)
        h_s_ = self.h_s_linear(h_s.permute(0, 2, 1))
        #(N, Hidden, L)
        h_t_ = self.h_t_linear(h_t).unsqueeze(2).expand(-1, -1, h_s_.size(2))

        # V 需要是（N, 1, Hidden） 乘出来是（N, 1, L）, 再经过softmax就是概率
        V = self.V.unsqueeze(0).unsqueeze(0).expand(h_s_.size(0), -1, -1)
        # (N, L)
        alpha = torch.bmm(V, self.tanh(h_s_ + h_t_)).squeeze(1)

        # 将访问过的City alpha 变为-inf再softmax
        self.inf = self._inf.unsqueeze(1).expand(alpha.size(0), alpha.size(1))
        # 不能  alpha[visted] = float('-inf')  谨防就地操作
        if len(alpha[visted]) > 0:
            alpha[visted] = self.inf[visted]

        alpha_ = self.softmax(alpha)
        # c_t = h_s * alpha_ (N, Hidden) 
        # 找到了，是attention层的内容
        c_t = torch.bmm(h_s_, alpha_.unsqueeze(2)).squeeze(2)
        return c_t, alpha_


In [6]:
# Decoder Model

class Decoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super(Decoder, self).__init__()

        self.embedding_dim = embedding_dim
        self.input0 = Parameter(torch.FloatTensor(embedding_dim), requires_grad=False)
        # decoder的input0初始化为[-1, 1]随机数 (batch, embedding_dim)
        nn.init.uniform_(self.input0, -1, 1)

        self.i2h = nn.Linear(embedding_dim, 4*hidden_dim)
        self.h2h = nn.Linear(hidden_dim, 4*hidden_dim)

        self.att = Attention(hidden_dim)

        # 为 0（False）代表没被访问过 
        self.visted = Parameter(torch.zeros(1), requires_grad=False) 

        # h ̃t = tanh(Wc[c_t; h_t])
        self.Wc = nn.Linear(2*hidden_dim, hidden_dim)

        # 辅助判断是否visted
        self.help_visted = Parameter(torch.zeros(1), requires_grad=False)

        self.sigmoid =  nn.Sigmoid()
        self.tanh =  nn.Tanh()


    def forward(self, embedded_inputs, decoder_h_0, decoder_c_0, encoder_output):
        batch_size = embedded_inputs.size(0)
        input_length = embedded_inputs.size(1)

        # (N, embedding)
        decoder_input = self.input0.unsqueeze(0).expand(batch_size, -1)
        
        # 初始 h, c 为encoder最后一个状态
        decoder_h = decoder_h_0
        decoder_c = decoder_c_0

        # (N, L)
        visted = self.visted.unsqueeze(0).expand(batch_size, input_length)

        # (N, L)
        help_visted = self.help_visted.repeat(input_length)
        for i in range(input_length):
            help_visted.data[i] = i
        help_visted = help_visted.unsqueeze(0).expand(batch_size, -1).long()


        # 最后要返回城市序列 pointers 和 每一级的output = alpha
        outputs = []
        pointers = []

        def step(x, h, c):
            """
            模拟一个lstm cell，然后增加一个layer层
            x: (N, embeding_dim)
            h: (N, Hidden)
            c: (N, Hidden)
            """ 
            # (N, 4 * Hidden)
            gates = self.i2h(x) + self.h2h(h)
            # (N, Hidden)
            input, forget, cell, out = gates.chunk(4,1)

            forget = self.sigmoid(forget)
            input = self.sigmoid(input)
            cell = self.tanh(cell)
            out = self.sigmoid(out)

            c_t = (c * forget) + (input * cell)
            # (N, Hidden)
            h_t = self.tanh(c_t) * out

            # h_s = encoder_output, h_t, visited 进入 attn 层， 获得c_t(N, Hidden), alpha (N, L)
            c_t, alpha = self.att(encoder_output, h_t, torch.eq(visted, 1))
            hidden_t = self.tanh(self.Wc(torch.cat((c_t, h_t), 1)))

            return hidden_t, c_t, alpha
        
        for _ in range(input_length):
            decoder_h, decoder_c, alpha = step(decoder_input, decoder_h, decoder_c)
            # 将访问过的alpha 置 0,然后求最大的alpha的坐标 indices(N)  
            # 不能alpha[torch.eq(visted, 1)] = 0，谨防就地操作
            alpha_ = alpha * (1 - visted)
            val, indices = alpha_.max(1)
            # alpha_ (N, L) 
            # 求得该次应该访问的城市，用indices(N),help_visted(N, L)更新visted (N,L),
            # tmp (N, L)
            tmp  = (help_visted == (indices.unsqueeze(1).expand(-1, input_length)))
            visted = 1 -  (1 - visted) * (1 - tmp.float())

            # 通过indeices(N), embedding_inputs(N, L, embedding_dim) 获得下一次的输入decoder_input(N, embedding)
            decoder_input = embedded_inputs[tmp.unsqueeze(2).expand(-1, -1, self.embedding_dim)].view(batch_size, self.embedding_dim)

            # 为了cat合并出想要的效果，这里需要做一些处理
            outputs.append(alpha_.unsqueeze(0))
            pointers.append(indices.unsqueeze(1))
        
        # (N, L, L（概率数组）) 表示每次的输出的alpha数组(L)， 每个aplha代表该次预测的各城市的概率
        outputs = torch.cat(outputs).permute(1, 0, 2)
        # (N, L) 表示一个最优解
        pointers = torch.cat(pointers, 1)

        return (outputs, pointers), (decoder_h, decoder_c)

In [7]:
class PtrNet(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_layers, dropout, bidirectional):
        """
        hidden_dim : encoder与decoder共用，隐状态feature数
        num_layers : encoder使用，拥用于init LSTM, 代表堆叠层数
        dropout : encoder使用，拥用于init LSTM, 代表丢弃数
        bidirectional : encoder使用，拥用于init LSTM, 代表是否双向输入
        """
        super(PtrNet, self).__init__()
        self.bidir = bidirectional
        self.embedding = nn.Linear(2, embedding_dim)
        self.encoder = Encoder(embedding_dim, hidden_dim, num_layers, dropout, bidirectional)
        self.decoder = Decoder(embedding_dim, hidden_dim)
    
    def forward(self, inputs):
        """ 
        inputs：(batch_size, seq_length, 2) 全部城市的坐标
        """
        batch_size = inputs.size(0)
        seq_length = inputs.size(1)
        
        # 对输入的数据进行embedding编码
        # (batch_size, seq_length, embedding_dim)
        embedded_inputs = self.embedding(  inputs.view(batch_size * seq_length, -1)   ).view(batch_size, seq_length, -1)

        #将embedded_inputs送入 encoder 中
        encoder_output, (encoder_h_n, encoder_c_n) = self.encoder(embedded_inputs)

        # decoder 需要的有：
        # encoder_output, (所有encoder的隐状态，用于传入attention)
        # decoder_h_0 = encoder_h_n, decoder_c_0 = encoder_c_n, 
        # seq_length循环次数,
        # embedded_inputs, 作为每次选出的城市，输入下一次循环中

        # embedded_inputs, encoder_h_n, encoder_c_n, encoder_output, seq_length
        if self.bidir:
            decoder_h = torch.cat( (encoder_h_n[-1],encoder_h_n[-2]), dim=-1)
            decoder_c = torch.cat( (encoder_c_n[-1],encoder_c_n[-2]), dim=-1)
        else:
            decoder_h = encoder_h_n[-1]
            decoder_c = encoder_c_n[-1]

        (outputs, pointers), (decoder_h, decoder_c) = self.decoder(embedded_inputs,
                                                           decoder_h, 
                                                           decoder_c,
                                                           encoder_output)
        return  outputs, pointers


In [28]:
train_size = 100000 # 训练集大小,每次epoch进行train_size个数据的训练
seq_length = 5  # 单个训练数据的城市数目

In [29]:
myDataset = TSP_Dataset(train_size , seq_length)

In [30]:
batch_size = 512  # batch大小，每epoch进行 train_size/batch_size 次batch  

In [31]:
myDataloader = DataLoader(dataset = myDataset, batch_size = batch_size, shuffle = True, num_workers = 2)

In [38]:
embedding_dim = 128  
hidden_dim = 512
num_layers = 4
dropout = 0.001
bidirectional = True 

In [39]:
mymodel = PtrNet(embedding_dim, hidden_dim, num_layers, dropout, bidirectional)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.


In [40]:
lr = 0.0005

In [41]:
CCE = torch.nn.CrossEntropyLoss()
mymodel_optim = optim.Adam(filter(lambda p: p.requires_grad, mymodel.parameters()), lr=lr)

In [42]:
num_epochs = 7  # epoch数

In [43]:
# 开始训练


losses = []
# writer = SummaryWriter()

for epoch in range(num_epochs):
    batch_loss = []
    iterator = tqdm(myDataloader, unit='Batch')
    for i, item in enumerate(iterator):
        iterator.set_description('Batch %i/%i' % (epoch+1, num_epochs))

        # torch.autograd.set_detect_anomaly(True)
        outputs, pointers = mymodel(item['Point'].float())
        # outputs (N, L, L)
        # item['Slove'] (N, L)


        # (N*L, L)
        outputs  = outputs.contiguous().view(-1, outputs.size()[-1])
        # (N*L)
        target = item['Slove'].view(-1)
        loss = CCE(outputs, target)
        
        mymodel_optim.zero_grad()
        loss.backward()
        mymodel_optim.step()

        losses.append(loss.data.item())
        batch_loss.append(loss.data.item())
        iterator.set_postfix(loss='{}'.format(loss.data.item()))

    iterator.set_postfix(loss=np.average(batch_loss))
    # writer.add_scalar('Loss/train', np.average(batch_loss), epoch)

# writer.close()

Batch 7/7: 100%|██████████| 196/196 [12:01<00:00,  3.68s/Batch, loss=1.1782735586166382]


In [24]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir '/content/runs/Oct29_03-30-08_287f258c494c'

按照参考代码中的预设跑出来的结果
```python
train_size = 10000 # 训练集大小,每次epoch进行train_size个数据的训练
seq_length = 5  # 单个训练数据的城市数目
batch_size = 256  # batch大小，每epoch进行 train_size/batch_size 次batch  
embedding_dim = 128  
hidden_dim = 512
num_layers = 2
dropout = 0.
bidirectional = True 
lr = 0.0001
num_epochs = 10  # epoch数

Batch 1/10: 100%|██████████| 40/40 [00:37<00:00,  1.06Batch/s, loss=1.5213171243667603]
Batch 2/10: 100%|██████████| 40/40 [00:38<00:00,  1.05Batch/s, loss=1.4382966756820679]
Batch 3/10: 100%|██████████| 40/40 [00:37<00:00,  1.05Batch/s, loss=1.4567874670028687]
Batch 4/10: 100%|██████████| 40/40 [00:37<00:00,  1.06Batch/s, loss=1.4119609594345093]
Batch 5/10: 100%|██████████| 40/40 [00:38<00:00,  1.05Batch/s, loss=1.426945686340332]
Batch 6/10: 100%|██████████| 40/40 [00:37<00:00,  1.06Batch/s, loss=1.4063466787338257]
Batch 7/10: 100%|██████████| 40/40 [00:37<00:00,  1.05Batch/s, loss=1.3852460384368896]
Batch 8/10: 100%|██████████| 40/40 [00:37<00:00,  1.06Batch/s, loss=1.3605930805206299]
Batch 9/10: 100%|██████████| 40/40 [00:38<00:00,  1.05Batch/s, loss=1.3718546628952026]
Batch 10/10: 100%|██████████| 40/40 [00:38<00:00,  1.05Batch/s, loss=1.2640124559402466]
```

和上次比调大了lr，loss降低了一点点

```python
train_size = 10000 # 训练集大小,每次epoch进行train_size个数据的训练
seq_length = 5  # 单个训练数据的城市数目
batch_size = 256  # batch大小，每epoch进行 train_size/batch_size 次batch  
embedding_dim = 128  
hidden_dim = 512
num_layers = 2
dropout = 0.
bidirectional = True 
lr = 0.001
num_epochs = 10  # epoch数


Batch 1/10: 100%|██████████| 40/40 [00:37<00:00,  1.06Batch/s, loss=1.4535272121429443]
Batch 2/10: 100%|██████████| 40/40 [00:38<00:00,  1.04Batch/s, loss=1.4366843700408936]
Batch 3/10: 100%|██████████| 40/40 [00:37<00:00,  1.06Batch/s, loss=1.2903785705566406]
Batch 4/10: 100%|██████████| 40/40 [00:37<00:00,  1.06Batch/s, loss=1.3250709772109985]
Batch 5/10: 100%|██████████| 40/40 [00:38<00:00,  1.05Batch/s, loss=1.2955352067947388]
Batch 6/10: 100%|██████████| 40/40 [00:38<00:00,  1.05Batch/s, loss=1.2900669574737549]
Batch 7/10: 100%|██████████| 40/40 [00:37<00:00,  1.06Batch/s, loss=1.2359356880187988]
Batch 8/10: 100%|██████████| 40/40 [00:37<00:00,  1.05Batch/s, loss=1.2447574138641357]
Batch 9/10: 100%|██████████| 40/40 [00:38<00:00,  1.05Batch/s, loss=1.2044010162353516]
Batch 10/10: 100%|██████████| 40/40 [00:38<00:00,  1.05Batch/s, loss=1.2382856607437134]
```

增加了训练集size,loss降低还算明显,运行时长853s

```python
train_size = 100000 # 训练集大小,每次epoch进行train_size个数据的训练
seq_length = 5  # 单个训练数据的城市数目
batch_size = 512  # batch大小，每epoch进行 train_size/batch_size 次batch  
embedding_dim = 128  
hidden_dim = 512
num_layers = 2
dropout = 0.
bidirectional = True 
lr = 0.0005
num_epochs = 10  # epoch数

Batch 1/10: 100%|██████████| 196/196 [05:59<00:00,  1.83s/Batch, loss=1.3405646085739136]
Batch 2/10: 100%|██████████| 196/196 [06:07<00:00,  1.87s/Batch, loss=1.2688865661621094]
Batch 3/10: 100%|██████████| 196/196 [07:22<00:00,  2.26s/Batch, loss=1.2330338954925537]
Batch 4/10: 100%|██████████| 196/196 [09:54<00:00,  3.03s/Batch, loss=1.1934809684753418]
Batch 5/10: 100%|██████████| 196/196 [10:51<00:00,  3.32s/Batch, loss=1.2616503238677979]
Batch 6/10: 100%|██████████| 196/196 [10:45<00:00,  3.30s/Batch, loss=1.1770641803741455]
Batch 7/10: 100%|██████████| 196/196 [10:49<00:00,  3.32s/Batch, loss=1.1408429145812988]
Batch 8/10: 100%|██████████| 196/196 [10:58<00:00,  3.36s/Batch, loss=1.155600666999817]
Batch 9/10: 100%|██████████| 196/196 [10:54<00:00,  3.34s/Batch, loss=1.1500990390777588]
Batch 10/10: 100%|██████████| 196/196 [10:57<00:00,  3.35s/Batch, loss=1.1630727052688599]

```